In [ ]:
!pip install napari
!pip install SimpleITK

In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%gui qt
import os
if 'TEST_ENV' in os.environ:
    TEST_ENV = os.environ['TEST_ENV'].lower() == "true"
else:
    TEST_ENV = 0
print(f"Running test environment: {bool(TEST_ENV)}")

Running test environment: False


In [ ]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

resp = urlopen("http://www.fmrib.ox.ac.uk/primers/intro_primer/ExBox3/ExBox3.zip")
zipfile = ZipFile(BytesIO(resp.read()))

img_file = zipfile.extract("ExBox3/T1_brain.nii.gz")
mask_file = zipfile.extract("ExBox3/T1_brain_seg.nii.gz")

In [2]:
import SimpleITK as sitk
import numpy as np

# load image and mask
img_file = "./ExBox3/T1_brain.nii.gz"
mask_file = "./ExBox3/T1_brain_seg.nii.gz"
img = sitk.GetArrayFromImage(sitk.ReadImage(img_file))
img = img.astype(np.float32)
mask = mask = sitk.GetArrayFromImage(sitk.ReadImage(mask_file))
mask = mask.astype(np.float32)

assert mask.shape == img.shape
print(f"Image shape {img.shape}")
print(f"Image shape {mask.shape}")

Image shape (192, 192, 174)
Image shape (192, 192, 174)


In [3]:
if TEST_ENV:
    def view_batch(batch):
        pass
else:
    %gui qt
    import napari
    def view_batch(batch):
            viewer = napari.view_image(batch["data"].cpu().numpy(), name="data")
            viewer.add_image(batch["mask"].cpu().numpy(), name="mask", opacity=0.2)

/Users/micha/miniconda3/envs/phoenix/lib/python3.7/site-packages/qtpy/__init__.py:216: RuntimeWarning: Selected binding "pyqt5" could not be found, using "pyside2"
  'using "{}"'.format(initial_api, API), RuntimeWarning)


In [4]:
import torch
from rising.transforms import *

batch = {
    "data": torch.from_numpy(img).float()[None, None],
    "mask": torch.from_numpy(mask).long()[None, None],
}

def apply_transform(trafo, batch):
    transformed = trafo(**batch)
    print(f"Transformed data shape: {transformed['data'].shape}")
    print(f"Transformed mask shape: {transformed['mask'].shape}")
    print(f"Transformed data min: {transformed['data'].min()}")
    print(f"Transformed data max: {transformed['data'].max()}")
    print(f"Transformed data mean: {transformed['data'].mean()}")
    return transformed

In [5]:
print(f"Transformed data shape: {batch['data'].shape}")
print(f"Transformed mask shape: {batch['mask'].shape}")
print(f"Transformed data min: {batch['data'].min()}")
print(f"Transformed data max: {batch['data'].max()}")
print(f"Transformed data mean: {batch['data'].mean()}")

Transformed data shape: torch.Size([1, 1, 192, 192, 174])
Transformed mask shape: torch.Size([1, 1, 192, 192, 174])
Transformed data min: 0.0
Transformed data max: 502.0
Transformed data mean: 37.62009048461914


In [8]:
trafo = Scale(1.5, adjust_size=False)
transformed = apply_transform(trafo, batch)
view_batch(transformed)

Transformed data shape: torch.Size([1, 1, 192, 192, 174])
Transformed mask shape: torch.Size([1, 1, 192, 192, 174])
Transformed data min: 0.0
Transformed data max: 445.4209289550781
Transformed data mean: 110.88246154785156


In [ ]:
trafo = Rotate([0, 0, 45], degree=True, adjust_size=False)
transformed = apply_transform(trafo, batch)
view_batch(transformed)

In [ ]:
trafo = Translate([0.1, 0, 0], adjust_size=False)
transformed = apply_transform(trafo, batch)
view_batch(transformed)

In [ ]:
trafo = CenterCropGrid(100)
transformed = apply_transform(trafo, batch)
view_batch(transformed)

In [ ]:
trafo = RandomDistortion(noise_type="normal", noise_kwargs={"mean": 0.1, "std": 0.005})
transformed = apply_transform(trafo, batch)
view_batch(transformed)

In [48]:
trafo = RadialDistortion(scale=[0.1, 1., 0.1])
transformed = apply_transform(trafo, batch)
view_batch(transformed)

tensor(0.4000)
tensor(0.0002)
{(1, 1, 192, 192, 174): tensor([[[[[-0.5966, -0.5969, -0.5969],
           [-0.5927, -0.5999, -0.5999],
           [-0.5887, -0.6029, -0.6029],
           ...,
           [ 0.5887, -0.6029, -0.6029],
           [ 0.5927, -0.5999, -0.5999],
           [ 0.5966, -0.5969, -0.5969]],

          [[-0.5993, -0.5934, -0.5996],
           [-0.5954, -0.5964, -0.6027],
           [-0.5914, -0.5994, -0.6057],
           ...,
           [ 0.5914, -0.5994, -0.6057],
           [ 0.5954, -0.5964, -0.6027],
           [ 0.5993, -0.5934, -0.5996]],

          [[-0.6020, -0.5898, -0.6024],
           [-0.5981, -0.5927, -0.6054],
           [-0.5940, -0.5957, -0.6084],
           ...,
           [ 0.5940, -0.5957, -0.6084],
           [ 0.5981, -0.5927, -0.6054],
           [ 0.6020, -0.5898, -0.6024]],

          ...,

          [[-0.6020,  0.5898, -0.6024],
           [-0.5981,  0.5927, -0.6054],
           [-0.5940,  0.5957, -0.6084],
           ...,
           [ 0.5940,